### Introduction

In this notebook, we explore the **Birthday Paradox**, explain the nature of the apparent paradox, and touch upon related concepts. The Birthday Paradox has important applications in cryptography and collision-based problems, such as hash functions and Simon's Problem.

### Table of Contents

1. **Statement of the Birthday Paradox**
2. **The Paradox**
3. **Birthday Paradox Approximation**
4. **Related Questions**
5. **References and Suggested Resources**

---

### Statement of the Birthday Paradox

> Given a group of $n$ people, what is the probability that at least two of them share the same birthday?


We assume: 

 - All 365 days are equally likely (ignoring leap years)
 - Birthdays are independent and uniformly distributed. 

To answer the birthday paradox, we first compute the probability of the complement: the probability that no two people share the same birthday. 

Let 
 - $B_i$ be the event that the $i^{\text{th}}$ person's birthday does not match any of the previous $(i-1)$ birthdays
 - Then the full event of interest is $A = B_1 \cap B_2 \cap \dots \cap B_n$, where all birthdays are different.

Using the chain rule of probability, we have:
$$
\begin{aligned}
P(A) &= P(B_1 \cap B_2 \cap \dots \cap B_n) \\
&= P(B_1) \cdot P(B_2 \mid B_1) \cdot P(B_3 \mid B_1 \cap B_2) \cdots P(B_n \mid B_1 \cap \dots \cap B_{n-1})
\end{aligned}
$$

Now we are going to compute these probabilities: 

 - $P(B_1)$: out of the 365 days in a year, any of the 365 days can be chosen, therefore $B_1 = \frac{365}{365}$.
 - $P(B_2)$: one birthday is taken, 364 remain out of 365: $P(B_2 | B_1) = \frac{364}{365}$
 - $P(B_3)$: two birthday is taken, 363 remain out of 365: $P(B_3 | B_1 \cap B_2 ) = \frac{363}{365}$
 - ...
 - $P(B_i | B_1 \cap B_2 \cap \dots B_{k-1}) = \frac{365 - (i-1) }{365}$


So the total probability of no collisions is: 

$$
P(n) = P\left(\bigcap\limits_{i=1}^{n} B_i\right) = \prod\limits_{i=1}^{n} \frac{365 - (i-1)}{365} = \frac{365}{365} \cdot \frac{364}{365} \cdot \frac{363}{365} \cdot \cdots \cdot \frac{365-n+1}{365}
$$

Hence, the probability that at least two people share a birthday is: $1 - P(n)$. 

$$
\DeclareMathOperator{\argmin}{argmin}
$$

Next we want to find the smallest value of $n$ such that the collision probability exceeds $50\%$: 

$$\argmin\limits_{n \in \mathbb{N}} \left\{ 1 - P(n) \geq \frac{1}{2} \right\}.$$

In [1]:
def birthday_prob(n):
    prob_unique = 1.0
    for k in range(n):
        prob_unique *= (365 - k) / 365
    return 1 - prob_unique

n = 1
while birthday_prob(n) < 0.5:
    n += 1

print(f"The smallest n such that P(n) >= 0.5 is {n}")


The smallest n such that P(n) >= 0.5 is 23


---
### The Paradox

The fact that with only 23 people there is more than $50\%$ probability of finding a collision in birthdays is rather counterintuitive. At first glance, one might naturally think that $\left\lceil \frac{365}{2} \right\rceil = 183$ people are needed to expect a match. Hence, the name "birthday paradox". 

However, there are alternative perspectives that help make sense of the seemingly surprising result. While only $23$ people are required to expect a collision, we are not just making $22$ comparisons. In fact, each new individual added to the group is compared against all the previous ones. Therefore with $23$ people, the number of comparisons is: 
$$
22+21+\cdots+1 = \frac{(23)(22)}{2} = {23 \choose 2} = 253. 
$$ 

In this light, maybe $23$ people is not that surprising now. 

---
### Birthday Paradox Approximation 

We have established that: 

$$P(n) = P \left( \bigcap\limits_{i=1}^n B_i \right) = \prod\limits_{i=1}^n \frac{365 - (i-1)}{365} = \prod\limits_{i=0}^{n-1} \frac{365 - i}{365} = \prod\limits_{i=0}^{n-1} 1 - \frac{i}{365}.$$

In the Birthday Paradox problem, $365$ represents the number of days in a year. More generally, in collision-finding problems, the domain size can be any value $N$. Thus, we generalize the expression as:

$$P(n) = \prod\limits_{i=0}^{n-1} 1 - \frac{i}{N}.$$


To find an approximation, we recognize that $0 \leq \frac{i}{N} \leq 1$ for $0 \leq i \leq n \leq N$, and that is the hint to apply the Taylor expansion of $\ln(1-x)$, which converges when $0 \leq x \leq 1$ to find an approximation of no collision in a collision-finding problem. 

Hence, 
$$
\begin{aligned}
&P(n) = \prod\limits_{i=0}^{n-1} 1 - \frac{i}{N} \\
\Rightarrow &\ln ( P(n) ) = \ln \left( \prod\limits_{i=0}^{n-1} 1 - \frac{i}{N} \right)
= \sum\limits_{i=0}^{n-1} \ln \left( 1 - \frac{i}{N}  \right)
\end{aligned}
$$

The Taylor series of $\ln(1 - x)$ is:


$$\ln(1-x) = -x - \frac{x^2}{2} - \frac{x^3}{3} - \dots.$$ 

Using the first-order approximation, we take: 
$$\ln \left( 1 - x \right) \approx -x$$. 

Therefore, 
$$
\begin{aligned}
\ln(P(n)) &= \sum\limits_{i=0}^{n=1} \ln \left( 1 - \frac{i}{N} \right) \\
&\approx \sum\limits_{i=0}^{n-1} -\frac{i}{N} \\
&= - \frac{1}{N} \sum\limits_{i=0}^{n-1} i \\
&= -\frac{1}{N} \cdot \frac{(n)(n-1)}{2} \quad \quad \quad \quad \quad \quad \text{since $\sum\limits_{i=0}^{n-1} i = 0+1+2+\dots+(n-1) = \frac{n(n-1)}{2}$}
\end{aligned}
$$

Additionally, for large $n$, we approximate $n(n-1) \approx n^2$. Hence, 
$$
\begin{aligned}
& \ln(P(n)) \approx -\frac{n^2}{2N} \\
\xRightarrow{\text{exponentiating both sides}} & P(n) \approx e^{-\frac{n^2}{2N}}
\end{aligned}
$$

Therefore, the probability of finding at least one collision is $1 - P(n) = 1 - e^{-\frac{n^2}{2N}}$.

To expect a collision, we solve for when the collision probability exceeds $50\%$. That is, 
$$1 - P(n) \geq \frac{1}{2}.$$

Using the approximation $P(n) \approx e^{- \frac{n^2}{2N}}$, we get: 

$$
\begin{aligned}
&1 - P(n) \geq \frac{1}{2} \\
\Rightarrow &1 - e^{-\frac{n^2}{2N}} \geq \frac{1}{2} \\ 
\Rightarrow &e^{-\frac{n^2}{2N}} \leq \frac{1}{2} \\
\Rightarrow &\frac{n^2}{2N} \geq \ln(2) \\
\Rightarrow &n^2 \geq 2N \ln(2) \\
\Rightarrow &n \geq \sqrt{ 2N \ln(2) }. 
\end{aligned}
$$

So for instance, in the classic birthday paradox problem, where $N=365$, we have 
$$\sqrt{2 \times 365 \times \ln(2)} \approx 22.33.$$

Taking the ceiling, $\lceil 22.33 \rceil = 23$. Thus, we approximately require $23$ people for there to be a greater than $50\%$ chance of a shared birthday.  

---
### Related Questions

We briefly review a few related questions to the Birthday Problem to highlight how the answers can vary by simply tweaking the question:

1. What's the probability that at least two people share a specific birthday, like January 1st?


We compute the probability that 0 or 1 person has January 1st, then subtract from 1.

- No one has Jan 1st:

$$
P_0 = \left( \frac{364}{365} \right)^n
$$

- Exactly one person has Jan 1st:

$$
P_1 = n \cdot \left( \frac{1}{365} \right) \cdot \left( \frac{364}{365} \right)^{n - 1}
$$

- At least two people have Jan 1st:

$$
P_{\geq 2} = 1 - P_0 - P_1
$$

In [3]:
def P_ge_2(n):
    P0 = (364 / 365) ** n
    P1 = n * (1 / 365) * (364 / 365) ** (n - 1)
    return 1 - P0 - P1

n = 1
while True:
    if P_ge_2(n) >= 0.5:
        break
    n += 1

print(f"Minimum n such that P(>= 2 people share Jan 1st) >= 0.5 is: {n}")


Minimum n such that P(>= 2 people share Jan 1st) >= 0.5 is: 613


Insight

- Birthday Paradox (any date): 23 people for >50% chance.
- Specific date (e.g., Jan 1st): Needs 613 people for similar chance.

2. How many people are needed for the probability that at least 3, 4, 5, etc. people share the same birthday to exceed $50\%$?

> The first few values are as follows: >50% probability of 3 people sharing a birthday - 88 people; >50% probability of 4 people sharing a birthday - 187 people. [1]

3. **Strong Birthday Problem**

What is the minimum number of people needed so that there is at least a 50% chance that every person shares their birthday with at least one other person?

>  For d=365 days the answer is 3,064 people. [1]

---

### References and Suggested Resources

[1] [Birthday Problem – Wikipedia](https://en.wikipedia.org/wiki/Birthday_problem)

[2] [Birthday Paradox, Factorial Approximations & Laplace Method – ekamperi.github.io](https://ekamperi.github.io/mathematics/2019/11/09/birthday-paradox-factorial-approximations-laplace-method.html)

[3] [Approximating the Birthday Problem – PlanetMath](https://planetmath.org/approximatingthebirthdayproblem).  

[4] [The Birthday Problem – University of Regina (PDF)](https://uregina.ca/~kozdron/Teaching/UBC/302Fall10/Handouts/birthday_problem.pdf).  

